In [66]:
# pip install datasets
from datasets import load_dataset


In [67]:
dataset_abstractive = load_dataset("sobamchan/aclsum", "abstractive", split="test")
dataset_extractive = load_dataset("sobamchan/aclsum", "extractive", split="test")

In [68]:
dataset_abstractive_10 = dataset_abstractive[:10]
dataset_extractive_10 = dataset_extractive[:10]

Let's import **ROUGE**

In [69]:
#pip install evaluate absl-py nltk rouge-score
from evaluate import load

In [70]:
rouge = load("rouge")

In [71]:
# Possibilities for rouge parameter are:  "rouge1", "rouge2", "rougeL"
def rouge_score(candidate, reference, rougex):
    result = rouge.compute(
        predictions=[candidate],
        references=[reference],
        rouge_types=[rougex]
    )
    r1 = result[rougex]
    return r1

### Let's define funcitons for **Summary-level's** strategies
#### **Greedy Search**


In [72]:
def greedy_extractive_summary(sentences, abstractive_summaries, eval_criteria, max_sent):
    selected = []
    remaining = sentences[:]
    #mask = [0] * len(sentences)

    while remaining and max_sent != 0: 
        best_sentence = None
        best_score = -1
        for sent in remaining:
            
            candidate_summary = " ".join(selected + [sent])
            r1 = rouge_score(candidate_summary, abstractive_summaries, eval_criteria)
            if r1 > best_score:
                best_score = r1
                best_sentence = sent
       
        selected.append(best_sentence)
        idx = sentences.index(best_sentence)
        #mask[idx] = 1
        remaining.remove(best_sentence)
        max_sent -= 1
    
    #return selected, mask
    return selected

#### **Beam Search**

In [73]:
def beam_extractive_summary(sentences, abstractive_summary, eval_criteria, max_sent, beam_size=4,):
    # Tuple of two items:
    #   -   selected sentences array
    #   -   ROUGE score
    beams = [([], 0.0)]
    
    for _ in range(max_sent):
        new_beams = []
        for selected, _ in beams:
            remaining = [s for s in sentences if s not in selected]
            
            for sent in remaining:
                candidate_summary = " ".join(selected + [sent])
                r1 = rouge_score(candidate_summary, abstractive_summary, eval_criteria)
                
                new_beams.append((selected + [sent], r1))
        
        # Sort by third value (ROUGE F1 score), highest score first
        new_beams.sort(key=lambda x: x[1], reverse=True)
        # Takes only the first beam_size elements
        beams = new_beams[:beam_size]
    
    return [beam for beam, _ in beams]

### Let's define funcitons for **Sentence-level's** strategies
#### **Local Scorer**


In [74]:
def local_extractive_summary(sentences, abstractive_summaries, eval_criteria, max_sent):
    def rank_sentences():
        sent_with_score = []
        for sent in sentences:
            r1 = rouge_score(sent, abstractive_summaries, eval_criteria)
            sent_with_score.append({"sentence": sent, "r_score": r1})
        sent_with_score.sort(key=lambda x: x["r_score"], reverse=True)
        return [sent["sentence"] for sent in sent_with_score]

    sorted_sentences = rank_sentences()
    return sorted_sentences[:max_sent]

#### **Global Scorer**
As heuristic in order to find the best candidates we will use **Beam Search** heuristic.

In [75]:
def global_extractive_summary(sentences, abstractive_summary, eval_criteria, max_sent, beam_size=4):
    candidates = beam_extractive_summary(sentences, abstractive_summary, eval_criteria, max_sent, beam_size)
    unique_sent_in_candidates = list(set([sent for candidate in [beam[0] for beam in candidates] for sent in candidate]))
    unique_sent_with_scores = []

    for sent in unique_sent_in_candidates:
        final_score = 0
        for cand, score in candidates:
            if sent in cand:
                final_score += score
        unique_sent_with_scores.append((sent, final_score))
    
    sorted_unique_sent_with_scores = sorted(unique_sent_with_scores, key=lambda x: x[1], reverse=True)

    best_sentences = [sent for sent, _ in sorted_unique_sent_with_scores][:max_sent]

    return best_sentences

### Let's now define a function to iterate over all documents
It will also keep track of the **time** taken from each algorithm in order to compute all the results. 

In [76]:
import time

In [77]:
def apply_heuristic_to_dataset(heuristic_fn, docs_sentences, abstractive_summaries, eval_criteria, max_sent=6, beam_size=0):
    selected_list = []
    start_time = time.perf_counter()
    for sentences, abs_summary in zip(docs_sentences, abstractive_summaries):
        if beam_size > 0:
            selected = heuristic_fn(sentences, abs_summary, eval_criteria, max_sent, beam_size)
        else:
            selected = heuristic_fn(sentences, abs_summary, eval_criteria, max_sent)
            
        selected_list.append(selected)

    elapsed_time = time.perf_counter() - start_time

    return selected_list, elapsed_time

### Let's first apply each heuristic once for each summary (**Challenge**, **Approach**, **Outcome**) 

As evaluation criteria we will first use **ROUGE F-1**

In [78]:
evaluation_criteria = "rouge1"

Since we want to grid search in the interval [1, 32] in order to determine which value for **max_sent** is the best, we will use 32. We will then use a beam size of 4 to avoid huge complexity.

In [79]:
max_sentences = 32
beam_sie = 4

In [80]:
# Take the list of all source sentences from extractive dataset
#list_source_sentences = list(dataset_extractive["source_sentences"])
list_source_sentences = list(dataset_extractive_10["source_sentences"])


#### **Challenge** summaries

In [81]:
# Take the list of all challenge sentences from abstractive dataset
#list_challenge_summaries = list(dataset_abstractive["challenge"])
list_challenge_summaries = list(dataset_abstractive_10["challenge"])

In [ ]:
# Greedy Search
greedy_list_summaries_challenge, greedy_challenge_time = apply_heuristic_to_dataset(greedy_extractive_summary, list_source_sentences, list_challenge_summaries, evaluation_criteria, max_sentences)

In [ ]:
# Beam Search
beam_list_summaries_challenge, beam_challenge_time = apply_heuristic_to_dataset(beam_extractive_summary, list_source_sentences, list_challenge_summaries, evaluation_criteria, max_sentences, beam_sie)

In [ ]:
# Local Scorer
local_list_summaries_challenge, local_challenge_time = apply_heuristic_to_dataset(local_extractive_summary, list_source_sentences, list_challenge_summaries, evaluation_criteria, max_sentences)

In [ ]:
# Global Scorer
global_list_summaries_challenge, global_challenge_time  = apply_heuristic_to_dataset(global_extractive_summary, list_source_sentences, list_challenge_summaries, evaluation_criteria, max_sentences, beam_sie)

### Evaluation

Let's make a function which, for each document, concatenates the sentences we have extracted from the selected list

In [ ]:
def sentence_concatenation(sentence_list):
    merged_sentences = [" ".join(inner_list) for inner_list in sentence_list]
    return merged_sentences

#### THE TWO CELLS BELOW ARE **TEMPORARY**

In [ ]:
beam_list_summaries_challenge = beam_list_summaries_challenge[0]

In [ ]:
beam_list_summaries_challenge = [beam for beam, _ in beam_list_summaries_challenge]

In [ ]:
# Generated summaries concatenation
greedy_list_summaries_challenge_conc = sentence_concatenation(greedy_list_summaries_challenge)
beam_list_summaries_challenge_conc = sentence_concatenation(beam_list_summaries_challenge)
local_list_summaries_challenge_conc = sentence_concatenation(local_list_summaries_challenge)
global_list_summaries_challenge_conc = sentence_concatenation(global_list_summaries_challenge)

# Labels concatenation
#labels_challenge = sentence_concatenation(dataset_extractive["challenge_labels"])
labels_challenge = sentence_concatenation(dataset_extractive_10["challenge_sentences"])

We now need the function in order to evaluate the average performances over the whole test set, base on **ROUGE F1**

In [ ]:
def avg_performance(predicted_summaries, label_summaries, eval_criteria):
    # In case something went wrong
    if len(predicted_summaries) != len(label_summaries):
        return None
    
    r1_sum = 0
    for pred, label in zip(predicted_summaries, label_summaries):
        r1 = rouge_score(pred, label, eval_criteria)
        r1_sum += r1
    
    return r1_sum / len(predicted_summaries)

In [ ]:
r1_greedy_challenge = avg_performance(greedy_list_summaries_challenge_conc, labels_challenge, evaluation_criteria)
print(f'The average ROUGE F-1 score for Greedy Search heuristic is: {round(r1_greedy_challenge, 5)}')
print(f'Time taken: {round(greedy_challenge_time, 1)}s')

The average ROUGE F-1 score for Greedy Search heuristic is: 0.00183
Time taken: 69.5s
